In [ ]:
from dotenv import load_dotenv

load_dotenv()    # Load API keys from secret .env file

True

### Define a context schema

In [2]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

### Creating an Agent with a "Context Schema"

In [10]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    context_schema=ColourContext  
)

### Passing the Context

In [4]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [5]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='b2908b8d-df54-46b5-8051-391db5647d7e'),
              AIMessage(content='I don’t know your favourite color. I don’t have access to your personal data. If you’d like, we can figure it out together with a quick quiz, or you can just tell me what you’re leaning toward.\n\nWant to try a short 3-question mini-quiz? Answer with A or B for each:\n1) Cool colors (A) or warm colors (B)?\n2) Bright/vivid (A) or soft/muted (B)?\n3) Color that goes with most outfits (A) or one that really stands out (B)?\n\nIf you give me your choices, I’ll suggest a color and a quick reason. Or tell me a color you’re already thinking of and I’ll help you confirm it.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2398, 'prompt_tokens': 12, 'total_tokens': 2410, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_t

### Accessing Context

In [6]:
from langchain.tools import tool, ToolRuntime

@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the favourite colour of the user"""
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the least favourite colour of the user"""
    return runtime.context.least_favourite_colour

### Giving the Tools to Agent

In [7]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

### Agent uses Tools to read Context

In [8]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

pprint(response)

/Users/nguyenanloc/PycharmProjects/Langchain/Langchain Foundation/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=ColourContext(favourite_c...vourite_colour='yellow'), input_type=ColourContext])
  return self.__pydantic_serializer__.to_python(


{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='9bd6b66b-4df1-434d-908a-31acd9c98925'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 276, 'prompt_tokens': 149, 'total_tokens': 425, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D46McHvzpmc7s0vRBFB9sM4YAyC0U', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c147b-3a6c-7f00-ab67-2b01bd6687c9-0', tool_calls=[{'name': 'get_favourite_colour', 'args': {}, 'id': 'call_WOl6AQtOHDdOpJji2XNjoKjD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 276, 'total_to

### Overriding Context (Custom Settings)

In [11]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    # This time we change the "ID Card" to say "green" instead of "blue"
    context=ColourContext(favourite_colour="green")
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='64f10779-d5d7-47e4-9d45-11da6b47824f'),
              AIMessage(content='I don’t know your favourite colour yet. I don’t have access to your preferences unless you tell me.\n\nIf you want, we can figure it out quickly:\n- Do you prefer cool colors (blue/green/purple) or warm colors (red/orange/yellow)?\n- Do you like bright, saturated shades or softer pastels?\n- Do you lean toward nature-inspired colors (blue/green/brown) or bold fashion colors (hot pink, neon, electric blue)?\n\nAnswer these and I’ll guess your likely favourite color. Or just tell me the color you already know you like.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1538, 'prompt_tokens': 12, 'total_tokens': 1550, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1408, 'rejected_prediction_tokens': 0}, 'pr